# Live video feed test with slowfast : LAB 1 Bonus

**TODO : Run as a .py script to check if it runs better with openCV**

> Author : Hugo BABIN-RIBY

Meant to run on windows.

Runs on CPU !

In [1]:
%pip install opencv-python
%pip install --upgrade pip
%pip install torch
%pip install pytorchvideo
%pip install --upgrade pytorchvideo
%pip install torchvision


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
^C
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import cv2
import torch

cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("Error: Could not open webcam")
try:
    while True:
        ret, frame = cap.read()
        print(torch.tensor(frame))
        cv2.imshow('Webcam Feed', frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break  
finally:
    cap.release()
    cv2.destroyAllWindows()

tensor([[[ 95, 125, 136],
         [ 94, 123, 134],
         [ 94, 121, 132],
         ...,
         [132, 158, 188],
         [129, 157, 192],
         [125, 154, 191]],

        [[ 95, 124, 135],
         [ 96, 124, 135],
         [ 98, 125, 136],
         ...,
         [132, 157, 187],
         [131, 159, 194],
         [131, 159, 196]],

        [[ 97, 124, 135],
         [100, 127, 138],
         [103, 128, 140],
         ...,
         [136, 159, 191],
         [136, 161, 197],
         [136, 162, 199]],

        ...,

        [[ 83, 118, 135],
         [ 83, 118, 136],
         [ 81, 115, 134],
         ...,
         [251, 251, 251],
         [251, 251, 251],
         [251, 251, 251]],

        [[ 82, 118, 135],
         [ 82, 118, 135],
         [ 81, 117, 135],
         ...,
         [251, 251, 251],
         [251, 251, 251],
         [251, 251, 251]],

        [[ 79, 116, 131],
         [ 80, 116, 133],
         [ 80, 116, 133],
         ...,
         [251, 251, 251],
        

In [3]:
# import slowfast
import torch
# Choose the `slowfast_r50` model 
model = torch.hub.load('facebookresearch/pytorchvideo', 'slowfast_r50', pretrained=True)


Using cache found in C:\Users\babin/.cache\torch\hub\facebookresearch_pytorchvideo_main


In [4]:
from typing import Dict
import json
import urllib
from torchvision.transforms import Compose, Lambda
from torchvision.transforms._transforms_video import (
    CenterCropVideo,
    NormalizeVideo,
)
from pytorchvideo.data.encoded_video import EncodedVideo
from pytorchvideo.transforms import (
    ApplyTransformToKey,
    ShortSideScale,
    UniformTemporalSubsample,
    UniformCropVideo
) 

device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)
model = model.eval()
model = model.to(device)

C:\Users\babin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torchvision\transforms\_functional_video.py:6: UserWarning: The 'torchvision.transforms._functional_video' module is deprecated since 0.12 and will be removed in the future. Please use the 'torchvision.transforms.functional' module instead.
  warnings.warn(
C:\Users\babin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torchvision\transforms\_transforms_video.py:22: UserWarning: The 'torchvision.transforms._transforms_video' module is deprecated since 0.12 and will be removed in the future. Please use the 'torchvision.transforms' module instead.
  warnings.warn(


cuda


In [5]:
json_url = "https://dl.fbaipublicfiles.com/pyslowfast/dataset/class_names/kinetics_classnames.json"
json_filename = "kinetics_classnames.json"
try: urllib.URLopener().retrieve(json_url, json_filename)
except: urllib.request.urlretrieve(json_url, json_filename)

with open(json_filename, "r") as f:
    kinetics_classnames = json.load(f)

# Create an id to label name mapping
kinetics_id_to_classname = {}
for k, v in kinetics_classnames.items():
    kinetics_id_to_classname[v] = str(k).replace('"', "")


In [6]:
side_size = 256
mean = [0.45, 0.45, 0.45]
std = [0.225, 0.225, 0.225]
crop_size = 256
num_frames = 32
sampling_rate = 2
frames_per_second = 30
slowfast_alpha = 4
num_clips = 10
num_crops = 3

class PackPathway(torch.nn.Module):
    """
    Transform for converting video frames as a list of tensors. 
    """
    def __init__(self):
        super().__init__()
        
    def forward(self, frames: torch.Tensor):
        fast_pathway = frames
        # Perform temporal sampling from the fast pathway.
        slow_pathway = torch.index_select(
            frames,
            1,
            torch.linspace(
                0, frames.shape[1] - 1, frames.shape[1] // slowfast_alpha
            ).long(),
        )
        frame_list = [slow_pathway.unsqueeze(0), fast_pathway.unsqueeze(0)]
        return frame_list

transform=Compose(
        [
            UniformTemporalSubsample(num_frames),
            Lambda(lambda x: x/255.0),
            NormalizeVideo(mean, std),
            ShortSideScale(
                size=side_size
            ),
            CenterCropVideo(crop_size),
            PackPathway()
        ]
    )

# The duration of the input clip is also specific to the model.
clip_duration = (num_frames * sampling_rate)/frames_per_second


In [9]:
import cv2
import torch
import time
from torchvision import transforms
from collections import deque
import numpy as np

device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

class WebcamProcessor:
    def __init__(self, model, device, transform, buffer_size=32):
        self.model = model
        self.device = device
        self.transform = transform
        self.buffer_size = buffer_size
        self.frame_buffer = deque(maxlen=buffer_size)
        self.pred_text = ""
        
    def process_frame(self, frame):
        """Convert OpenCV BGR frame to tensor and normalize"""
        return torch.tensor(frame)
        
    def process_buffer(self):
        """Process the accumulated frames with the model"""
        frames = torch.stack(list(self.frame_buffer))
        frames = frames.permute((3,0,1,2))
        # print(frames.shape) # Dimensions : [C,T,H,W]
        inputs = self.transform(frames)
        inputs = [i.to(device) for i in inputs]
        # print(inputs[0].shape, inputs[1].shape)
        with torch.no_grad():
            predictions = self.model(inputs)
        # clear the buffer
        del self.frame_buffer
        self.frame_buffer = deque(maxlen=self.buffer_size)
        return predictions
    
    def create_text_display(self, text, width=500, height=50):
        # Create a black image for text
        text_display = np.zeros((height, width, 3), np.uint8)
        
        # Add text to the image
        font = cv2.FONT_HERSHEY_COMPLEX
        font_scale = 0.5
        font_thickness = 1
        font_color = (255, 255, 255)  # White text
        
        # Calculate text size and position to center it
        (text_width, text_height), _ = cv2.getTextSize(text, font, font_scale, font_thickness)
        text_x = (width - text_width) // 2
        text_y = (height + text_height) // 2
        
        cv2.putText(text_display, text, (text_x, text_y), font, font_scale, font_color)
        return text_display
        
    def start_capture(self):
        cap = cv2.VideoCapture(0)
        
        if not cap.isOpened():
            print("Error: Could not open webcam")
            return
        
        frame_counter = 0
        
        try:
            while True:
                ret, frame = cap.read()

                if frame_counter % 2 == 0: #process 1/2 of frames, more or less, depends on your computing power
                    processed_frame = self.process_frame(frame)
                    self.frame_buffer.append(processed_frame)
                    
                    # Process buffer if we have enough frames
                    if len(self.frame_buffer) == self.frame_buffer.maxlen:
                        predictions = self.process_buffer()
                        # Get the predicted classes
                        post_act = torch.nn.Softmax(dim=1)
                        preds = post_act(predictions)
                        pred_classes = preds.topk(k=3).indices[0]

                        # Map the predicted classes to the label names
                        pred_class_names = [kinetics_id_to_classname[int(i)] for i in pred_classes]
                        self.pred_text = ("T3 preds: %s" % ", ".join(pred_class_names))
                
                # Create text display
                frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
                text_display = self.create_text_display(self.pred_text, width=frame_width)
                
                # Combine frame and text display vertically
                combined_display = np.vstack((frame, text_display))
                
                # Show combined display
                cv2.imshow('Webcam Feed with Predictions', combined_display)
                frame_counter += 1
                
                if cv2.waitKey(1) & 0xFF == ord('q'):
                    break
                    
        finally:
            cap.release()
            cv2.destroyAllWindows()

model = model.to(device) 
print(device)
model.eval()

processor = WebcamProcessor(
    model=model,
    device=device,
    transform=transform,
    buffer_size=num_frames
)

# Start processing
processor.start_capture()

cuda
cuda
